In [1]:
%%time 

import pandas as pd
import os
#this makes your code more universal. Allows us to arrive at the working directory of whoever wants to run this script t
path = os.getcwd() 

# the line below assumes whoever is running this script has the data in question loaded in a demo_data folder in their working directory
data = pd.read_csv(path + '/demo_data/dcard-top100.csv')

# similarly, there should be a list of stop words in the demo_data folder that this code can read in
read_stoplist = open(path +'/demo_data/stopwords/tomlinNTUB-chinese-stopwords.txt', encoding = "utf-8")

#remove \n lines from stops
stoplist = [line.rstrip('\n') for line in read_stoplist]

#preparing our preprocessing tools
import ckip_transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import unicodedata
ws_driver = CkipWordSegmenter(level=3, device=-1) #note - 'level=1' will run much faster, so initialize level 1 while still experimenting with code

#here, we have a function that will take the data, run in through a pipeline, and yield our desired results. 
def text_freq_counter(text):  
    lemmas = []
    for i in text:
        #The line below removes all non-chinese characters
        non_chin_rem = [''.join([c for c in i if unicodedata.category(c)[:2] in ["Lo"]])]
        #Because of the above line of code, some of the content was completely erased leaving empty lists
        #The following for-loop skips over any empty iterables (iterables = rows in the column of data)
        for i in non_chin_rem:
            if i == "":
                continue
            #now we can segment our data with the ckip transformer
            ws = ws_driver(non_chin_rem)
            #the segmenter created a lot of nested lists. the line below unnests them.
            flat = [i for j in ws for i in j]
            #remove the stop words from our list
            stop_rem = [i for i in flat if i not in stoplist]
            # put all segmented words into a list
            lemmas.append(stop_rem)
        #again, we have a list of lists, so we unnest them
        flat_lemmas = [i for j in lemmas for i in j]
        #filter for bigrams
        bigrams = [i for i in flat_lemmas if len(i) == 2]
        #create a data frame
        df_lem = pd.DataFrame(bigrams, columns = ['LEMMAS'])        
        #get frequency counts
        freq = df_lem["LEMMAS"].value_counts().reset_index()
        #name columns
        freq.columns = ["LEMMAS","FREQ"]
    #final object is a dataframe of bigram frequencies
    return freq

# create a variable that save the data we run through the function we created above
Q4_1 = text_freq_counter(data.content)
#print our results
Q4_1.head(21)

Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.47s/it]


Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.75it/s]


Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.19s/it]

Wall time: 2min 11s


,LEMMAS,FREQ
0,真的,116
1,沒有,93
2,覺得,90
3,知道,70
4,看到,67
5,現在,63
6,喜歡,56
7,朋友,55
8,一直,52
9,其實,52


In [2]:
%%time

#initialize the named entity recognition segmenter. Again, notice the level
ner_driver = CkipNerChunker(level=3, device=-1)

#create a function below that will yield our desired results
def ner_freq(data):
    ner_list = []
    for i in data:
        #remove symbols
        cleaned = [''.join([c for c in i if unicodedata.category(c)[0] not in ["S"]])]
        #extract named entities
        ner = ner_driver(cleaned, use_delim = True)
        #put into empty list prepared before the current for-loop
        ner_list.append(ner)
    #results from previous for-loop yielded nested lists within lists. Function belows flattens them.
    unnest = [i for j in ner_list for i in j]
    #use a for-loop to extract the named entities from within the sentences
    ner_ext = []
    for sentence_ner in unnest:
        for entity in sentence_ner:
            ner_ext.append(entity)
    pre_ner_df = []
    #filter the named entities to only include the locations and organizations 
    for i in ner_ext:
        if i[1] == 'GPE':
            pre_ner_df.append(i[0])
        if i[1] == 'ORG':
            pre_ner_df.append(i[0])
    #create data frame and get frequencies
    df = pd.DataFrame(pre_ner_df, columns = ['Named Entities'])
    freq = df["Named Entities"].value_counts().reset_index()
    freq.columns = ['Named Entities', 'Freq']
    return freq

Q4_2 = ner_freq(data.content)
Q4_2.head(21)

Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.18s/it]


Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.97s/it]

Wall time: 6min 24s


,Named Entities,Freq
0,台灣,23
1,日本,18
2,台南,7
3,台,6
4,英國,5
5,台中,4
6,韓國,4
7,沖繩,4
8,聖圭,4
9,德國,3


In [3]:
%%time

#import necessary modules and tools for processing
import spacy

#the code below resets the default stop list in spacy to match the one provided by Alvin 
custom_stopwd = set(stoplist)
override = spacy.util.get_lang_class('zh')
override.Defaults.stop_words = custom_stopwd
from spacy.lang.zh.stop_words import STOP_WORDS
nlp = spacy.load('zh_core_web_trf')

#create function to produce freqency count of bigram nouns and verbs
def nvfreq(data):
    #strip out non-chinese characters
    chin_only = [''.join([c for c in i if unicodedata.category(c)[:2] in ["Lo"]]) for i in data]
    #ust spacy parser
    n = [nlp(i) for i in chin_only]
    toks = []
    #extract nouns and verbs
    for lists in n:
        for tokens in lists:
            if tokens.pos_ == "NOUN":
                toks.append(tokens)
            if tokens.pos_ == "VERB":
                toks.append(tokens)
    #filter extracted nouns and verbs so that they are only bigrams
    bigram = [i for i in toks if len(i)==2]
    #remove stops utilizing the stopwords we replaced with the default earlier
    filt_bigram = [i for i in bigram if i.is_stop==False]
    #extract just the strings, because the elements in the filt_bigram list are spacy tokens (differnt kind of object that cannot be processed with the counting method below. 
    count_obj = [i.text for i in filt_bigram]
    #create a dataframe and produce frequencies
    df = pd.DataFrame(count_obj, columns = ['N-V'])
    freq = df["N-V"].value_counts().reset_index()
    freq.columns = ['N-V', 'FREQ']
    return freq

#Run data through function and produce results 
Q4_3 = nvfreq(data.content)
print(Q4_3.head(21))

#generate a wordcloud 
from wordcloud import WordCloud
vapor = Q4_3.set_index('N-V').to_dict()['FREQ']
cloud = WordCloud().generate_from_frequencies(vapor)
image = cloud.to_image()
image.show()

   N-V  FREQ
0   知道    70
1   看到    67
2   喜歡    55
3   朋友    54
4   分享    43
5   男友    42
6   工作    40
7   感覺    38
8   沒有    38
9   希望    35
10  發現    34
11  今天    33
12  時間    33
13  感情    33
14  蛋糕    31
15  出去    30
16  想要    30
17  公司    29
18  問題    28
19  故事    27
20  關係    27
Wall time: 1min 52s


In [4]:
%%time

#create a function that will produce a subj-pred frequency data frame
def depfreq(data):
    #strip away alphanumerics and symbols
    chin_only = [''.join([c for c in i if unicodedata.category(c)[:2] not in ["L", "S", "Lu", "Ll"]]) for i in data]
    #parse
    n = [nlp(i) for i in chin_only]
    s_p = []
    #extract nsubj and concatenate them with the head
    for i in n:
        for sents in i:
            if sents.dep_ == "nsubj":
                x = sents.text + '_' + sents.head.text
                s_p.append(x)
    #create df and do freq counts
    df = pd.DataFrame(s_p, columns = ['SUBJ-PRED'])
    freq = df["SUBJ-PRED"].value_counts().reset_index()
    freq.columns = ['SUBJ-PRED', 'FREQ']
    return freq

#run function with data and view results
Q4_4 = depfreq(data.content)
Q4_4.head(21)

Wall time: 1min 57s


,SUBJ-PRED,FREQ
0,我_喜歡,21
1,我_想,20
2,他_說,20
3,我_覺,19
4,我_知道,16
5,我_看,14
6,我_看到,11
7,我_愛,10
8,我_用,9
9,大家_好,8
